# Generating Useful Wikidata Files

This notebook generates files that contain derived data that is useful in many applications. The input to the notebook is the full Wikidata or a subset of Wikidata. It also works for arbutrary KGs as long as they follow the representation requirements of Wikidata:

- the *instance of* relation is represented using the `P31` property
- the *subclass of* relation is represented using the `P279` property
- all properties declare a datatype, and the data types must be one of the datatypes in Wikidata.

Inputs:

- `claims_file`: contains all statements, which consist of edges `node1/label/node2` where `label` is a property in Wikidata (e.g., sitelinks, labels, aliases and description are not in the claims file.
- `item_file`: the subset of the `claims_file` consistin of edges for property of data type `wikibase-item`
- `label_file`, `alias_file` and `description_file` containing labels, aliases and descriptions. It is assume that these files contain the labels, aliases and descriptions of all nodes appearing in the claims file. Users may provide these files for specific languages only.

Outputs:

- **Instance of (P31):** `derived.P31.tsv.gz` contains all the `instance of (P31)` edges present in the claims file.
- **Subclass of (P279):** `derived.P279.tsv.gz` contains all the `subclass of (P279)` edges present in the claims file.
- **Is A (isa):** `derived.isa.tsv.gz` contains edges `node`isa/node2` where either `node1/P31/node2` or `node1/P279/node2`
- **Closure of subclass of (P279star):** `derived.P279star.tsv.gz` contains edges `node1/P279star/node2` where `node2` is reachable from `node1` via zero or more hops using the `P279` property. Note that for example, `Q44/P279star/Q44`. An example when this file is useful is when you want to find all the instance of a class, including instances of subclasses of the given class.
- **In/out degrees:** `metadata.out_degree.tsv.gz` contains the out degree of every node, and `metadata.in_degree.tsv.gz` contains the in degree of every node.
- **Pagerank:** outputs page rank on the directed graph in `metadata.pagerank.directed.tsv.gz` and page rank of the directed graph in `metadata.pagerank.undirected.tsv.gz`.

### Batch Invocation
Example batch command. The second argument is a notebook where the output will be stored. You can load it to see progress.

```
papermill Wikidata\ Useful\ Files.ipynb useful-files.out.ipynb \
-p claims_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v4/all.tsv.gz \
-p label_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v4/part.label.en.tsv.gz \
-p item_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v4/part.wikibase-item.tsv.gz \
-p property_item_file = /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v4/part.property.wikibase-item.tsv.gz \
-p output_path <local folder> \
-p output_folder useful_files_v4 \
-p temp_folder temp.useful_files_v4 \
-p delete_database no 
-p languages es,ru,zh-cn
```

In [ ]:
# Parameters

# Folder on local machine where to create the output and temporary folders
# output_path = "/Users/pedroszekely/Downloads/kypher"
output_path = "/nas/home/mbmann/subset/output"

# The names of the output and temporary folders
# output_folder = "useful_wikidata_files_v4"
# temp_folder = "temp.useful_wikidata_files_v4"
temp_folder = "temp.useful_files"
output_folder = "useful_files"

# The location of input files
# wiki_root_folder = "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20200803-v4/"
wiki_root_folder = "/nas/home/mbmann/subset/output/parts/"

claims_file = "claims.tsv.gz"
label_file = "labels.en.tsv.gz"
alias_file = "aliases.en.tsv.gz"
description_file = "descriptions.en.tsv.gz"
item_file = "claims.wikibase-item.tsv.gz"

label_all = "labels.tsv.gz"
alias_all = "aliases.tsv.gz"
description_all = "descriptions.tsv.gz"

# Location of the cache database for kypher
# cache_path = "/Users/pedroszekely/Downloads/kypher/temp.useful_wikidata_files_v4"
# cache_path = "/nas/home/mbmann/subset/output"
cache_path = None

# Whether to delete the cache database
delete_database = False

# Whether to compute pagerank as it may not run on the laptop
compute_pagerank = True
languages = 'en'

# shortcuts to commands
kgtk_path = "~/anaconda3/envs/kgtk-subset/bin/kgtk"
kgtk = f'time {kgtk_path} --debug'

In [ ]:
languages = languages.split(',')

In [ ]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd

import altair as alt

## Set up environment and folders to store the files

- `OUT` folder where the output files go
- `TEMP` folder to keep temporary files , including the database
- `kgtk` shortcut to invoke the kgtk software
- `kypher` shortcut to invoke `kgtk query with the cache database
- `CLAIMS` the `all.tsv` file of wikidata that contains all edges except label/alias/description
- `LABELS` the file with the English labels
- `ITEMS` the wikibase-item file (currently does not include node1 that are properties so for now we need the net file
- `STORE` location of the cache file

**VERSION 1:** Use os.environ for setting paths

In [ ]:
# if cache_path:
#     os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(cache_path)
# else:
#     os.environ['STORE'] = "{}/{}/wikidata.sqlite3.db".format(output_path, temp_folder)
# os.environ['OUT'] = "{}/{}".format(output_path, output_folder)
# os.environ['TEMP'] = "{}/{}".format(output_path, temp_folder)
# os.environ['kgtk'] = f"{kgtk_path}"
# os.environ['kgtk'] = f"time {kgtk_path} --debug"

# os.environ['kypher'] = f"time {kgtk_path} --debug query --graph-cache " + os.environ['STORE']
# # os.environ['kypher'] = f"time {kgtk_path} --debug query"

# os.environ['CLAIMS'] = wiki_root_folder + claims_file
# os.environ['LABELS'] = wiki_root_folder + label_file
# os.environ['ALIASES'] = wiki_root_folder + alias_file
# os.environ['DESCRIPTIONS'] = wiki_root_folder + description_file
# os.environ['ITEMS'] = wiki_root_folder + item_file

**VERSION 2:** Use {} for setting paths

In [ ]:
if cache_path:
    store = "{}/wikidata.sqlite3.db".format(cache_path)
else:
    store = "{}/{}/wikidata.sqlite3.db".format(output_path, temp_folder)
out = "{}/{}".format(output_path, output_folder)
temp = "{}/{}".format(output_path, temp_folder)
kgtk = f"{kgtk_path}"
kgtk = f"time {kgtk_path} --debug"

kypher = f"time {kgtk_path} --debug query --graph-cache " + store

claims = wiki_root_folder + claims_file
labels = wiki_root_folder + label_file
aliases = wiki_root_folder + alias_file
descriptions = wiki_root_folder + description_file
items = wiki_root_folder + item_file

In [ ]:
# !$kgtk #Working

# !echo $kypher
# !time ~/anaconda3/envs/kgtk-subset/bin/kgtk --debug query -h #Working
# !time ~/anaconda3/envs/kgtk-subset/bin/kgtk --debug query \
# --graph-cache /nas/home/mbmann/subset/output/wikidata.sqlite3.db #Working

In [ ]:
# !$kgtk #Working
# !{kgtk_path} #Working

# !time ~/anaconda3/envs/kgtk-subset/bin/kgtk --debug query -h \
# --graph-cache $STORE #Working

# !time {kgtk} --debug query \
# --graph-cache $store #Not Working

Echo the variables to see if they are all set correctly

In [ ]:
!echo {out}
!echo {temp}
!echo {kgtk}
!echo {kypher}
!echo {claims}
!echo {labels}
!echo {aliases}
!echo {descriptions}
!echo {items}
!echo {store}
!alias col="column -t -s $'\t' "

Go to the output directory and create the subfolders for the output files and the temporary files

In [ ]:
cd $output_path

In [ ]:
!mkdir -p {out}
!mkdir -p {temp}

Clean up the output and temp folders before we start

In [ ]:
# !rm $OUT/*.tsv $OUT/*.tsv.gz
# !rm $TEMP/*.tsv $TEMP/*.tsv.gz

In [ ]:
if delete_database:
    print("Deleteddatabase") 
    !rm {store}

In [ ]:
!ls -l {out}
!ls {temp}
!ls -l {claims}
!ls -l {labels}
!ls -l {aliases}
!ls -l {descriptions}
!ls {store}

In [ ]:
!zcat < {claims} | head | col

### Preview the input files

It is always a good practice to peek a the files to make sure the column headings are what we expect

In [ ]:
!{kypher} -i {claims} --limit 10 | col 

Force creation of the index on the label column

In [ ]:
!{kypher} -i {claims} -o - \
--match '(i)-[:P31]->(c)' \
--limit 5 \
| column -t -s $'\t' 

Force creation of the index on the node2 column

In [ ]:
!{kypher} -i {claims} -o - \
--match '(i)-[r]->(:Q5)' \
--limit 5 \
| column -t -s $'\t' 

### Count the number of edges

Counting takes a long time

In [ ]:
!{kypher} -i {claims} \
--match '()-[r]->()' \
--return 'count(r) as count' \
--limit 10

### Get labels, aliases and descriptions for other languages

In [ ]:
for lang in languages:
    cmd = f"{kgtk_path} --debug query --graph-cache {store} -i {wiki_root_folder}{label_all} -o {output_path}/{output_folder}/labels.{lang}.tsv.gz --match '(n1)-[l:label]->(n2)' --where 'n2.kgtk_lqstring_lang_suffix = \"{lang}\"' --return 'n1, l.label, n2, l.id' "
    !{cmd}

In [ ]:
for lang in languages:
    cmd = f"{kgtk_path} --debug query --graph-cache {store} -i {wiki_root_folder}{alias_all} -o {output_path}/{output_folder}/aliases.{lang}.tsv.gz --match '(n1)-[l:alias]->(n2)' --where 'n2.kgtk_lqstring_lang_suffix = \"{lang}\"' --return 'n1, l.label, n2, l.id' "
    !{cmd}

In [ ]:
for lang in languages:
    cmd = f"{kgtk_path} --debug query --graph-cache {store} -i {wiki_root_folder}{description_all} -o {output_path}/{output_folder}/descriptions.{lang}.tsv.gz --match '(n1)-[l:description]->(n2)' --where 'n2.kgtk_lqstring_lang_suffix = \"{lang}\"' --return 'n1, l.label, n2, l.id' "
    !{cmd}

### Create the P31 and P279 files

Create the `P31` file

In [ ]:
!{kypher} -i {claims} -o {out}/derived.P31.tsv.gz \
--match '(n1)-[l:P31]->(n2)' \
--return 'l, n1, l.label, n2' 

In [ ]:
!zcat {out}/derived.P31.tsv.gz | head | col

Create the P279 file

In [ ]:
!{kypher} -i {claims} -o {out}/derived.P279.tsv.gz \
    --match '(n1)-[l:P279]->(n2)' \
    --return 'l, n1, l.label, n2' 

### Create the file that contains all nodes reachable via P279 starting from a node2 in P31 or a node1 in P279

First compute the roots

In [ ]:
!{kypher} -i {out}/derived.P279.tsv.gz -o {temp}/P279.n1.tsv.gz \
--match '(n1)-[l]->()' \
--return 'n1 as id' 

In [ ]:
!{kypher} -i {out}/derived.P31.tsv.gz -o {temp}/P31.n2.tsv.gz \
--match '()-[l]->(n2)' \
--return 'n2 as id' 

In [ ]:
!{kgtk} cat --mode NONE -i {temp}/P31.n2.tsv.gz {temp}/P279.n1.tsv.gz \
| gzip > {temp}/P279.roots.1.tsv.gz

In [ ]:
!{kgtk} sort2 --mode NONE --column id -i {temp}/P279.roots.1.tsv.gz \
| gzip > {temp}/P279.roots.2.tsv.gz

We have lots of duplicates

In [ ]:
!zcat < {temp}/P279.roots.2.tsv.gz | head

In [ ]:
!{kgtk} compact -i {temp}/P279.roots.2.tsv.gz --mode NONE \
    --presorted \
    --columns id \
> {temp}/P279.roots.tsv

Now we can invoke the reachable-nodes command

In [ ]:
!{kgtk} reachable-nodes \
    --rootfile {temp}/P279.roots.tsv \
    --selflink \
    -i {out}/derived.P279.tsv.gz \
| gzip > {temp}/P279.reachable.tsv.gz

In [ ]:
!zcat < {temp}/P279.reachable.tsv.gz | head | col

The reachable-nodes command produces edges labeled `reachable`, so we need one command to rename them.

In [ ]:
!{kypher} -i {temp}/P279.reachable.tsv.gz -o {temp}/P279star.1.tsv.gz \
--match '(n1)-[]->(n2)' \
--return 'n1, "P279star" as label, n2 as node2' 

Now we can concatenate these files to produce the final output

In [ ]:
!{kgtk} sort2 -i {temp}/P279star.1.tsv.gz -o {temp}/P279star.2.tsv.gz

Make sure there are no duplicates

In [ ]:
!{kgtk} compact --presorted -i {temp}/P279star.2.tsv.gz -o {temp}/P279star.3.tsv.gz

Add ids

In [ ]:
!{kgtk} add-id --id-style node1-label-node2-num -i {temp}/P279star.3.tsv.gz -o {out}/derived.P279star.tsv.gz

In [ ]:
!zcat < {out}/derived.P279star.tsv.gz | head | col

This is how we would do the typical `?item P31/P279* ?class` in Kypher. 
The example shows how to get all the counts of instances of subclasses of city (Q515).

In [ ]:
!{kypher} -i {out}/derived.P31.tsv.gz -i {out}/derived.P279star.tsv.gz -i {labels} \
--match 'P31: (n1)-[:P31]->(c), P279star: (c)-[]->(:Q515), label: (n1)-[:label]->(label), label: (c)-[:label]->(c_label)' \
--return 'distinct c as class, count(c) as count, c_label as `class name`, n1 as instance, label as `label`' \
--order-by 'count(c) desc, c, n1' \
--limit 10 \
| col

Illustrate that it is indeed `P279*`

In [ ]:
!{kypher} -i {out}/derived.P31.tsv.gz -i {out}/derived.P279star.tsv.gz -i {labels} \
--match 'P31: (n1)-[:P31]->(c), P279star: (c)-[]->(:Q63440326), label: (n1)-[:label]->(label), label: (c)-[:label]->(c_label)' \
--return 'distinct c as class, c_label as `class name`, n1 as instance, label as `label`' \
--order-by 'c, n1' \
--limit 10 \
| col 

Test that `P279star` is indeed star

In [ ]:
!{kypher} -i {out}/derived.P279star.tsv.gz \
--match '(n1:Q44)-[:P279star]->(n2:Q44)'

### Create a file to do generalized Is-A queries
The idea is that `(n1)-[:isa]->(n2)` when `(n1)-[:P31]->(n2)` or `(n1)-[:P279]->(n2)`

We do this by concatenating the files and renaming the relation

In [ ]:
!{kgtk} cat -i {out}/derived.P31.tsv.gz {out}/derived.P279.tsv.gz \
    | gzip > {temp}/isa.1.tsv.gz

In [ ]:
!{kypher} -i {temp}/isa.1.tsv.gz -o {out}/derived.isa.tsv.gz \
--match '(n1)-[]->(n2)' \
--return 'n1, "isa" as label, n2' 

Example of how to use the `isa` relation

In [ ]:
!{kypher} -i {out}/derived.isa.tsv.gz -i {out}/derived.P279star.tsv.gz -i {labels} -o - \
--match 'isa: (n1)-[l:isa]->(c), P279star: (c)-[]->(:Q44), label: (n1)-[:label]->(label)' \
--return 'distinct n1, l.label, "Q44" as node2, label as n1_label' \
--limit 10 \
| col

## Compute pagerank

Now compute pagerank. These commands will exceed 16GB memory for graphs containing over 25 million nodes.

In [ ]:
if compute_pagerank:
    !{kgtk} graph-statistics -i {items} -o {out}/metadata.pagerank.directed.tsv.gz \
    --page-rank-property directed_pagerank \
    --pagerank --statistics-only \
    --log {temp}/metadata.pagerank.directed.summary.txt 

In [ ]:
if compute_pagerank:
    !cat {temp}/metadata.pagerank.directed.summary.txt

In [ ]:
if compute_pagerank:
    !{kgtk} graph-statistics -i {items} -o {out}/metadata.pagerank.undirected.tsv.gz \
    --page-rank-property undirected_pagerank \
    --pagerank --statistics-only --undirected \
    --log {temp}/metadata.pagerank.undirected.summary.txt 

In [ ]:
if compute_pagerank:
    !cat {temp}/metadata.pagerank.undirected.summary.txt 

## Compute Degrees

Kypher can compute the out degree by counting the node2s for each node1

In [ ]:
!{kypher} -i {claims} -o {temp}/metadata.out_degree.tsv.gz \
--match '(n1)-[l]->()' \
--return 'distinct n1 as node1, count(distinct l) as node2, "out_degree" as label' 

In [ ]:
!{kgtk} add-id --id-style node1-label-node2-num -i {temp}/metadata.out_degree.tsv.gz \
-o {temp}/metadata.out_degree2.tsv.gz

! {kgtk} sort2 -i {temp}/metadata.out_degree2.tsv.gz -o {out}/metadata.out_degree.sorted.tsv.gz

In [ ]:
!zcat {out}/metadata.out_degree.sorted.tsv.gz | head #Working

To count the in-degree we only care when the node2 is a wikibase-item

In [ ]:
!zcat {items} |head

In [ ]:
#Get in-degree for each node2, only if node2 is a wikibase-item.
!{kypher} -i {items} -o {temp}/metadata.in_degree.tsv.gz \
--match '()-[l]->(n2)' \
--return 'distinct n2 as node1, count(distinct l) as node2, "in_degree" as label' \
--order-by 'n2'

In [ ]:
# !zcat {temp}/metadata.in_degree.tsv.gz |head

In [ ]:
# rename columns before adding ids, KGTK will complain
in_degree_file = f'{temp}/metadata.in_degree.tsv.gz'
in_degree2_file = f'{temp}/metadata.in_degree.1.tsv.gz'
df = pd.read_csv(in_degree_file, sep='\t')
df = df.rename(columns={"node2": "node1"})
df = df.rename(columns={'count(DISTINCT graph_1_c1."id")': "node2"})
df.to_csv(in_degree2_file, sep='\t', index=False)

In [ ]:
!{kgtk} add-id --id-style node1-label-node2-num -i {temp}/metadata.in_degree.tsv.gz \
-o {temp}/metadata.in_degree2.tsv.gz

!{kgtk} sort2 -i {temp}/metadata.in_degree2.tsv.gz -o {out}/metadata.in_degree.sorted.tsv.gz

In [ ]:
!zcat < {out}/metadata.in_degree.sorted.tsv.gz | head

Calculate the distribution so we can make a nice chart

In [ ]:
!{kypher} -i {out}/metadata.in_degree.sorted.tsv.gz -o {out}/statistics.in_degree.distribution.tsv \
--match '(n1)-[]->(n2)' \
--return 'distinct n2 as in_degree, count(distinct n1) as count, "count" as label' \
--order-by 'cast(n2, integer)' 

In [ ]:
!head {out}/statistics.in_degree.distribution.tsv | col

In [ ]:
!{kypher} -i {out}/metadata.out_degree.sorted.tsv.gz -o {out}/statistics.out_degree.distribution.tsv \
--match '(n1)-[]->(n2)' \
--return 'distinct n2 as out_degree, count(distinct n1) as count, "count" as label' \
--order-by 'cast(n2, integer)' 

Draw some charts

In [ ]:
data = pd.read_csv(
    out + "/statistics.in_degree.distribution.tsv", sep="\t"
)

alt.Chart(data).mark_circle(size=60).encode(
    x=alt.X("in_degree", scale=alt.Scale(type="log")),
    y=alt.Y("count", scale=alt.Scale(type="log"), title="count of nodes"),
    tooltip=["in_degree", "count"],
).interactive().properties(title="Distribution of In Degree")

In [ ]:
data = pd.read_csv(
    out + "/statistics.out_degree.distribution.tsv", sep="\t"
)

alt.Chart(data).mark_circle(size=60).encode(
    x=alt.X("out_degree", scale=alt.Scale(type="log")),
    y=alt.Y("count", scale=alt.Scale(type="log"), title="count of nodes"),
    tooltip=["out_degree", "count"],
).interactive().properties(title="Distribution of Out Degree")

## Summary of results

In [ ]:
!ls -lh {out}/*